In [1]:
from typing import Callable, Tuple, TypeVar, cast, get_origin, get_args, Union, Any, Optional, Dict, List
from components import EventBroadcaster, MessageChannel, create_component_starter, loop

from components.two_wheels import TwoWheelsV3
from components.logger import LoggerComponent
from components.controller.bluetooth_controller import BlueToothCarControlSPP_V2
from components.controller.bluetooth_SPP_server import start_bluetooth_server_v2
from components.gyroscope.gyroscope import AngularSpeedControlV3
from components.camera import Picamera2V2
from data_collection.data_collection import LoggerSet
from multiprocessing import Manager



import datetime
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)



%load_ext autoreload
%autoreload 2

In [2]:
loggerset = LoggerSet('../log/session'+str(datetime.datetime.now()), overwrite_ok=False)
manager = Manager()

## Two Wheels

In [3]:

logger_starter = create_component_starter(
    LoggerComponent, 
    manager, 
    loop,  # doesn't need a loop....
    init_kwargs=dict(
        loggerset = loggerset
    ),
    loop_kwargs={'ideal_interval': 1/100},
)

two_wheel_starter = create_component_starter(
    TwoWheelsV3, 
    manager, 
    loop, 
    init_kwargs=dict(
        left_pin = 33, 
        right_pin = 32, 
        dir_pins = (16, 18), 
        name='TwoWheelsV3'
    ),
    loop_kwargs={'ideal_interval': 1/100},
    instantiater = TwoWheelsV3.entry

)

angular_speed_control_starter = create_component_starter(
    AngularSpeedControlV3, 
    manager, 
    loop, 
    init_kwargs=dict(
        i2c_address=0x68, 
        bus_num=1,
        name='AngularSpeedControlV3'
    ),
    loop_kwargs={'ideal_interval': 1/100},
    instantiater = AngularSpeedControlV3.entry
)

bluetooth_control_starter = create_component_starter(
    BlueToothCarControlSPP_V2, 
    manager, 
    loop, 
    init_kwargs={},
    loop_kwargs={'ideal_interval': 1/100},
)


picamera_starter = create_component_starter(
    Picamera2V2, 
    manager, 
    loop, 
    init_kwargs=dict(
        resolution=(114, 64), 
        framerate=30,
        name='Picamera2V2'
    ),
    loop_kwargs={'ideal_interval': 1/30},
)

bt_ser_out, bt_ser = start_bluetooth_server_v2(manager)

Making discoverable...
Starting Serial Port Profile...
NewConnection(/org/bluez/hci0/dev_C8_BD_4D_BA_4F_15, 88)


In [4]:

logger_starter.register_incoming_events(
    dict(
        log_handler=[
            two_wheel_starter.outgoing_events['logging'], 
            angular_speed_control_starter.outgoing_events['logging'], 
            bluetooth_control_starter.outgoing_events['logging'], 
            picamera_starter.outgoing_events['logging'], 
        ]
    )
)



two_wheel_starter.register_incoming_samples(
    angular_speed_control_starter.outgoing_samples
)

angular_speed_control_starter.register_incoming_samples(
    bluetooth_control_starter.outgoing_samples
)

bluetooth_control_starter.register_incoming_samples(
    bt_ser_out
)

logger_starter.start()
two_wheel_starter.start()
angular_speed_control_starter.start()
bluetooth_control_starter.start()
picamera_starter.start()



/home/kawa/projects/car1/src/components/two_wheels.py:24: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:75: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[0], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:76: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[1], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/__init__.py:185: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")
/home/kawa/projects/car1/src/components/__init__.py:185: UserWarning: the real interval is >= ideal_interval
  warnings.warn("the real interval is >= ideal_interval")

saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Picamera2V2
saving: AngularSpeedControlV3
saving: TwoWheelsV3
saving: BlueToothCarControlSPP_V2
saving: Pi

In [5]:

logger_starter.process.terminate()
two_wheel_starter.process.terminate()
angular_speed_control_starter.process.terminate()
bluetooth_control_starter.process.terminate()
picamera_starter.process.terminate()



# ML Component


In [186]:
ml_out, ml_starter = default_component_process_starter_v2(
    ImageMLController, 
    init_kwargs= dict(
        #resolution=(114, 64), framerate=30,
        model_path='test.keras',
        logger_set=logger_set, 
        name='ImageMLController'
    ), 
    mainloop=default_loop_v2, 
    main_kwargs=dict(interval=1/60), 
    manager=manager, 
)
ml = ml_starter(cam_out)

In [194]:
ml.terminate()

In [3]:
d = {'a':1, 'b':False}

if b:= d.get('b'):
    print (b)